# KATYDID vs Local Hough Transformation

#### Spectrograms generated by LOCUST. Track analysis by KATYDID and Local Hough Transformation. 

In [1]:
import LocustFakeEvent
import h5py
import uproot
import matplotlib.pyplot as plt
import numpy as np
import numpy.ma as ma
import matplotlib.image as mpimg 
import math
import time
import sklearn
import scipy

In [2]:
working_dir = '../Locust_out'
katydid_config = '../Katydid_LOCUST_may_2019_config.yaml'
locust_config = '../LocustFakeTrack_distribution_choices.json'
sim_name = 'test_fake_event'
min_snr=0.0
snr=60.0
N=4096
SR=100e6
dt = 1.0/SR

In [3]:
pi=math.pi
tan=math.tan
cos=math.cos
sin=math.sin
exp=math.exp
sqrt=math.sqrt

In [98]:
start = time.time()
pixel_threshold=1#parameters used
angle_dim=20
window_size=5
n_std=1.5
th_consecutive=3
max_spacing=5
snr=60
min_snr=snr
    
img, frequencies, true_events = LocustFakeEvent.run_simulation(working_dir=working_dir, 
                                                                            sim_name=sim_name, 
                                                                            locust_config_path=locust_config, 
                                                                            snr=snr, 
                                                                            min_snr=min_snr, 
                                                                            N=N, SR=SR, fixed_snr=False)

katydid_events = LocustFakeEvent.run_analysis(working_dir, sim_name, katydid_config)

    
print("---------------LOCUST Simulation---------------")
print()
for i in range(len(true_events[0])):
    print("Statistics for track",(i+1))
    print(true_events[0][i])
    print("------------------------------------")
    print()
    
print("---------------KATYDID Analysis---------------")
print()
for i in range(len(katydid_events)):
    print("Statistics for track",(i+1))
    print(katydid_events[i])
    print("------------------------------------")
    print()
    
binary_img, binary_final = local_hough_track_reconstruction(img, window_size, angle_dim, pixel_threshold, n_std,th_consecutive,max_spacing)
start_x,start_y,end_x,end_y=get_track_stats(binary_final,window_size)
spec_final=get_spec_final(binary_final,img)

    
print("-----------Local Hough predictions--------------")
for track in range(len(start_x)):
    print("Statistics for track",(track+1))
    print("Starting freq:",abs((frequencies[start_y[track]]/(2*pi))+2.59*10**10))
    print("Starting time:",(N/SR)*start_x[track])
    print("Ending freq:",abs((frequencies[end_y[track]]/(2*pi))+2.59*10**10))
    print("Ending time:",(N/SR)*end_x[track])
    window=spec_final[start_x[track]-5:end_x[track]+5,start_y[track]-5:end_y[track]+5]
    slope,y_int=get_best_fit(window)
    #print("Predicted slope:",slope)
    print("------------------------------------")
    print()
        
end = time.time()
print(end-start, "seconds")

SNR from uniform dist: 60 - 60

Simulated SNR: 60.0


Copy locust config to sim path
LocustSim config=../Locust_out/test_fake_event/LocustFakeTrack_distribution_choices.json
	Created: ../Locust_out/test_fake_event/locust_wnoise.egg
	Locust simulation time was 3.107362747192383s
slices and bins 488 4096
[b'EventID', b'ntracks', b'StartFrequencies', b'StartTimes', b'EndTimes', b'TrackLengths', b'Slopes', b'LOFrequency', b'RandomSeed', b'TrackPower', b'PitchAngles']
Mix-down frequency: [2.59e+10]
	Running Katydid...
Katydid -c ../Locust_out/test_fake_event/Katydid_LOCUST_may_2019_config.yaml -e ../Locust_out/test_fake_event/locust_wnoise.egg --rtw-file ../Locust_out/test_fake_event/reconstructed_event.root
	Katydid time was 0.8350768089294434s
---------------LOCUST Simulation---------------

Statistics for track 1
{'start_time': 0.006504496921550892, 'end_time': 0.006726790964144274, 'start_freq': 25909842824.180805, 'slope': 0.3151590894673042}
------------------------------------

Stati

In [4]:
def get_spec_final(binary_final,img):
    spec_final=np.zeros(shape=(img.shape[0],img.shape[1]))
    for i in range(binary_final.shape[0]):
        for j in range(binary_final.shape[1]):
            if(binary_final[i,j]==True):
                spec_final[i,j]=img[i,j]
    return spec_final

In [5]:
def get_best_fit(window):
    x_pts=[]
    y_pts=[]
    num=0
    den=0
    sum=0
    y_int=0
    mean_x=0
    mean_y=0
    for i in range(window.shape[0]):
        for j in range(window.shape[1]):
            if(window[i,j]>0):
                x_pts.append(i)
                y_pts.append(j)
                
    if(len(x_pts)==0 or len(y_pts)==0):
        return -1,-1  
    else:
        mean_x=np.mean(x_pts)
        mean_y=np.mean(y_pts)
    
        for i in range(len(x_pts)):
            sum=sum+window[x_pts[i],y_pts[i]]
        I_avg=sum/len(x_pts)
        
        for n in range(len(x_pts)):
            num=num+(window[x_pts[n],y_pts[n]]**2)*(x_pts[n]-mean_x)*(y_pts[n]-mean_y)
        for n in range(len(x_pts)):
            den=den+(window[x_pts[n],y_pts[n]]**2)*(y_pts[n]-mean_y)**2
        slope=num/den
    
        y_int=mean_y-(slope*mean_x)
        return slope,y_int

In [25]:
def local_hough_track_reconstruction(img, window_size, angle_dim, pixel_threshold, n_std,th_consecutive,max_spacing):

    width = int(window_size/2)

    binary_img = get_binary_img(img,n_std)
    binary_final = np.zeros(shape=binary_img.shape, dtype=bool)
    
    angles = np.linspace(-np.pi/2,0,angle_dim)

    ind = get_window_centers(binary_img, width)
    
    x = np.arange(-width, width+1, 1)
    y = np.arange(-width, width+1, 1)
    xx, yy = np.meshgrid(x,y)

    for k in range(ind.shape[0]):

        i=ind[k,0]
        j=ind[k,1]

        x_min = i-width
        x_max = i+width+1
        y_min = j-width
        y_max = j+width+1
    
        window = binary_img[x_min:x_max,y_min:y_max]
        
        binary_result = get_voting(binary_img, xx, yy, window, angles)

        local_result = get_local_result(binary_result,pixel_threshold)
        
        #points_x,points_y,count=get_coordinates(local_result)
        
        #local_result_final=get_local_result_final(local_result,points_x,points_y,count,th_consecutive,max_spacing,width)

        binary_final[x_min:x_max,y_min:y_max] = binary_final[x_min:x_max,y_min:y_max]|local_result
    
    return binary_img,binary_final

In [7]:
def get_voting(img, xx, yy, window, angles):
    
    result=xx*np.cos(angles[:,None,None])+yy*np.sin(angles[:,None,None])

    voting_matrix = np.zeros(shape=result.shape, dtype=bool)
    
    bool_mask = window[None,:,:]&(np.abs(result)<0.5)

    voting_matrix[bool_mask]=1
    
    return voting_matrix

In [8]:
def get_local_result(binary_result, pixel_threshold):
    
    hist = np.sum(binary_result,axis=(1,2))
    #plt.plot(hist)
    #plt.show()
    result_angles = hist>pixel_threshold

    return np.any(binary_result[result_angles],axis=0)

In [9]:
def get_window_centers(binary_img, width):
    ind = np.argwhere(binary_img)
    ind_c=(ind[:,0]>=width)&(ind[:,0]<binary_img.shape[0]-width)&\
            (ind[:,1]>=width)&(ind[:,1]<binary_img.shape[1]-width)
    ind = ind[ind_c]
    return ind

In [84]:
def get_binary_img(img,n_std):
    threshold = np.mean(img)+np.std(img)
    binary_img = img>threshold
    return binary_img

In [11]:
def get_coordinates(local_result):
    
    max_points=10*local_result.shape[0]
    theta_dim=90
    x=int(local_result.shape[0]/2)
    y=int(local_result.shape[1]/2)
    points_x=np.zeros(shape=(theta_dim,max_points),dtype='int')
    points_y=np.zeros(shape=(theta_dim,max_points),dtype='int')
    count=np.zeros(shape=(theta_dim),dtype='int')
    
    for i in range(0,local_result.shape[0]):
        for j in range(0,local_result.shape[1]):
            for itheta in range(0,theta_dim,angle_dim):
                theta=(np.pi/180)*itheta
                result=(i-x)*np.cos(theta)+(j-y)*np.sin(theta)
                if (abs(result)<=0.5 and local_result[i,j]==True):
                    points_x[itheta,count[itheta]]=i
                    points_y[itheta,count[itheta]]=j
                    count[itheta]=count[itheta]+1
                    
    return points_x,points_y,count

In [12]:
def get_local_result_final(local_result,points_x,points_y,count,th_consecutive,max_spacing,size):
    theta_dim=90
    local_result_final=np.zeros(shape=(local_result.shape[0],local_result.shape[1]),dtype=bool)
    max_points=np.max(points_x)
    for itheta in range(theta_dim):
        points_x[itheta].sort()
        points_y[itheta].sort()
        label_min_x=0
        label_max_x=0
        label_max_x,label_min_x=is_continuous(points_x[itheta],max_points,max_spacing)
        if(label_max_x-label_min_x>=th_consecutive):
            for icount in range(label_min_x+1,label_max_x):
                delta_x=points_x[itheta,icount]-points_x[itheta,icount-1]
                delta_y=points_y[itheta,icount]-points_x[itheta,icount-1]
                
                if(delta_x==0 and delta_y==0):
                    local_result_final[points_x[itheta,icount-1]+size,points_y[itheta,icount-1]+size]=True 
                
                elif(delta_x>0 and delta_y==0):
                    for i in range(0,delta_x+1):
                        local_result_final[points_x[itheta,icount-1]+size+i,points_y[itheta,icount-1]+size]=True
                        
                elif(delta_x==0 and delta_y>0):
                    for i in range(0,delta_y+1):
                        local_result_final[points_x[itheta,icount-1]+size,points_y[itheta,icount-1]+size+i]=True
                        
                elif(delta_x>0 and delta_y>0):
                    slope=np.tan(delta_y/delta_x)
                    for i in range(0,delta_x+1):
                        local_result_final[points_x[itheta,icount-1]+size+i,points_y[itheta,icount-1]+size+int(i*slope)]=True
                                   
                
    return local_result_final

In [13]:
def is_continuous(coord_x,max_points,max_spacing):
    start=0
    
    if(np.sum(coord_x)>0):
        while(coord_x[start]==0):
            start=start+1
        
    label_min_x=start
    label_max_x=start+1 
    index_lower_x=label_min_x
    index_upper_x=label_max_x
    while(index_upper_x<max_points):
        if(coord_x[index_upper_x]-coord_x[index_upper_x-1]<=max_spacing): #or coord_x[index_upper_x]-coord_x[index_upper_x-1]==1 or coord_x[index_upper_x]-coord_x[index_upper_x-1]==2):
            if(index_upper_x-index_lower_x>label_max_x-label_min_x):
                label_min_x=index_lower_x
                label_max_x=index_upper_x-1
            index_upper_x=index_upper_x+1
            continue
        
        index_lower_x=index_lower_x+1
        index_upper_x=index_lower_x+1
    
    return label_max_x,label_min_x

In [94]:
def get_track_stats(binary_final,size):
    start_x=[]
    start_y=[]
    end_x=[]
    end_y=[]
    slope=[]
    vacant_row=1
    y=-1
    x=0
    x_ref=0
    while(y<binary_final.shape[1]-1):
        y=y+1
        track_pixel=0
        sum=np.sum(binary_final[:,y])
        if(sum>0):
            for x in range(int(size/2),binary_final.shape[0]-int(size/2)): #This is for checking the starting pixel coordinates of a track
                if(binary_final[x,y]==True):
                    window_sum=np.sum(binary_final[x-int(size/2):x+int(size/2),y-int(size/2):y+int(size/2)])
                    if(window_sum>=9):
                        track_pixel=1
                        start_x.append(x)
                        start_y.append(y)
                        break
                    else:
                        track_pixel=0
       # print(y)
        if(track_pixel==1):
            vacant_row_total=0
            while(y<binary_final.shape[0]-1):
                y=y+1
                if(np.sum(binary_final[:,y])>0):
                    for x in range(int(size/2),binary_final.shape[0]-int(size/2)): #This is for checking the ending pixel coordinates of a track
                    
                        if(binary_final[x,y]==True):
                            window_sum=np.sum(binary_final[x-int(size/2):x+int(size/2),y-int(size/2):y+int(size/2)])
                            #print("Window",window_sum)
                            if(window_sum>=9):
                                vacant_row=0  #This means that the bright pixels in a row are actually track pixels
                                x_ref=x
                                break
                            else:
                                vacant_row=1  #This means that the bright pixels in a row are actually noise
                #print(vacant_row)
                if(np.sum(binary_final[:,y])==0 or vacant_row==1):
                    vacant_row_total=vacant_row_total+1
                    vacant_row=1
                
                if(vacant_row_total==2):
                    #print(y-2)
                    end_y.append(y-2)
                    end_x.append(x_ref)
                    break    
                    
    
    start_x_new=[]
    start_y_new=[]
    for i in range(len(end_x)-1):
        start_x_new.append(start_x[i])
        start_y_new.append(start_y[i])
        
    return start_y_new,start_x_new,end_y,end_x